In [2]:
import os
import random
import sys
from typing import Optional, Callable, List, Dict, Tuple

import dgmc
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import torch
from torch import Tensor
import funcs
#import pytorch_lightning as pl
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric

%reload_ext autoreload
%autoreload 2

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
device

'cpu'

In [3]:
file_path = f"s3://drose-sandbox/sizmek_zync_cleaned_793k.csv"

df_raw = pd.read_csv(file_path, dtype="str")
print(f"Shape: {df_raw.shape[0]:,} Memory: {df_raw.memory_usage(deep=True).sum()/1e9:.2f}GB")

Shape: 793,073 Memory: 1.55GB


In [4]:
df_raw.head(1).T

,0
Unnamed: 0,0
account_id,19967
url,tvguide.com
city_code,5731371
state_code,OR
dma_code,820
country_code,US
user_id,5314600042548863
session_id,d1b6ae4f-7582-42d9-90bc-9b793bcc9ca7:162949223...
referrer,https://www.google.com/


In [5]:
sizmek_cols = ["user_id", "url", "account_id", "city_code",
               "state_code", "dma_code", "country_code", "sizmek_ip"]
zync_cols = ["session_id", "referrer", "client", "user_agent_platform",
             "user_agent_language", "user_agent_browser", "zync_ip", "zync_country",
             "zync_state", "zync_city", "zync_lat", "zync_long"]

#sizmek_cols = ["user_id", "url", "state_code", "country_code"]
#zync_cols = ["session_id", "referrer", "zync_state", "zync_country"]

df = df_raw[sizmek_cols + zync_cols]
print(df.shape)
#df = df[df[sizmek_cols + zync_cols].astype(bool)]  
#print(df.shape)
df = df.fillna("missing")
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df = df.reset_index(drop=True)
print(df.shape)

df = df.sample(n=100)

#for column in ["url", "referrer"]:
#    df[column] = df[column].apply(
#        lambda x:funcs.urlparse(x).netloc if pd.notnull(x) else x
#    )

group_list = [e for e in (sizmek_cols + zync_cols) if e not in ("url", "referrer")]
df = df.groupby(group_list)[["url", "referrer"]].agg(lambda x: list(x))
df = df.reset_index()
print(df.shape)
df= df[df["user_id"].duplicated(keep=False)]
print(df.shape)
df = df.drop("user_id", axis=1)

sizmek_features = ["url", "account_id", "city_code", "state_code",
               "dma_code", "country_code", "sizmek_ip"]
zync_features = ["referrer", "client", "user_agent_platform", "user_agent_language", 
             "user_agent_browser", "zync_ip", "zync_country",
             "zync_state", "zync_city", "zync_lat", "zync_long"]

sizmek_df = df[sizmek_features]
zync_df = df[zync_features]

(793073, 20)
(793073, 20)
(793073, 20)
(793073, 20)
(87, 20)
(14, 20)


In [6]:
sizmek_df.head(3)

,url,account_id,city_code,state_code,dma_code,country_code,sizmek_ip
18,[https://www.windstream.net/?domain=windstream...,19967,5074472,NE,652,US,70.182.33.29
19,[https://www.windstream.net/?domain=windstream...,35927,5074472,NE,652,US,70.182.33.29
52,[https://www.mor-tv.com/vuz],35927,4744870,VA,511,US,54.226.145.105


In [7]:
zync_df.head(3)

,referrer,client,user_agent_platform,user_agent_language,user_agent_browser,zync_ip,zync_country,zync_state,zync_city,zync_lat,zync_long
18,[https://www.windstream.net/?domain=windstream...,sizmek,windows,missing,chrome,70.182.33.29,US,NE,Omaha,41.2315,-96.0519
19,[https://www.windstream.net/?domain=windstream...,sizmek,windows,missing,chrome,70.182.33.29,US,NE,Omaha,41.2315,-96.0519
52,[https://www.thenation.com/],sizmek,android,missing,chrome,3.95.205.25,US,VA,Ashburn,39.0481,-77.4728


In [8]:
y_base = list(range(0,30_000))
print(y_base[:4])
random.shuffle(y_base)
print(y_base[:4])

[0, 1, 2, 3]
[10068, 27670, 19868, 7280]


# Define Dataset

In [9]:
class ZetaDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root: str, sizmek_df: pd.DataFrame, zync_df: pd.DataFrame, column: str, 
                 label: str, feature_cols=None, parse_url=False, expand_x=None, transform=None, 
                 pre_transform=None
        ):
        self.root = root
        self.sizmek_df = sizmek_df
        self.zync_df = zync_df
        self.column = column
        self.label = label
        self.feature_cols = feature_cols
        self.parse_url = parse_url
        self.expand_x = expand_x
        super(ZetaDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        print("Removing processed file. . .")
        os.remove(self.processed_paths[0])

    @property
    def raw_file_names(self) -> List[str]:
        return [
            "sizmek_bidstream_raw_20210625_10k.csv", 
            "zync_session_tracking_orc_20210625_10k.csv"
        ]

    @property
    def processed_file_names(self):
        return ["ZetaDataset.pt"]
    
    @property
    def edge_count_1(self):
        return zeta_data.data.edge_index1.shape[1]
    
    @property
    def edge_count_2(self):
        return zeta_data.data.edge_index2.shape[1]

    def download(self):
        pass

    def process(self):
        print("Creating Sizmek graph. . .")
        x1, edge_index1 = self.process_graph(
            self.sizmek_df, 
            self.column[0], 
            self.feature_cols["sizmek"]["categorical"],
            expand_x=256
        )
        print("Creating Zync graph. . .")
        x2, edge_index2 = self.process_graph(
            self.zync_df, 
            self.column[1], 
            self.feature_cols["zync"]["categorical"], 
            expand_x=256
        )

        train_y, test_y = self.process_y()

        data = Data(x1=x1, edge_index1=edge_index1, x2=x2,
                    edge_index2=edge_index2, train_y=train_y,
                    test_y=test_y)
        torch.save(self.collate([data]), self.processed_paths[0])

    def process_graph(self, df, column: str, feature_cols: List, expand_x: int=None):
        #df = pd.read_csv(file_path, low_memory=False)
        #df.columns = [i.split(".")[1] for i in df.columns]

        # parse URLs
        if self.parse_url == True and column in ["url", "referrer"]:
            print("Parsing URL")
            df[column] = df[column].apply(
                lambda x:funcs.urlparse(x).netloc if pd.notnull(x) else x
            )

        # Encode features
        print("Encoding features")
        feature_enc = OneHotEncoder(handle_unknown="ignore")
        features = pd.DataFrame(
            feature_enc.fit_transform(df[feature_cols]).toarray(), 
            columns=feature_enc.get_feature_names(feature_cols)
        )
        
        if expand_x is not None:
            print(f"Expanding X to {expand_x}")
            new_cols = [f"fake_{expand_x-i}" for i in range(expand_x - features.shape[1])][::-1]
            for col in new_cols:
                features[col] = 0
        x = torch.tensor(features.values, dtype=torch.float)

        # Create edge list
        print(f"Creating edge list on {column}")
        blocklist = funcs.count_edge_frequency(df, column)
        print(f"Unique URLs: {len(blocklist):,}")
        
        blocklist = blocklist[blocklist["count"] > 50]["url"]
        print(f"Common URLs blocked: {len(blocklist):,}")

        edges = funcs.connect_edges(df, column, blocklist)
        edge_index = torch.tensor(
            edges[["source", "target"]].T.values, dtype=torch.long
        )

        return x, edge_index

    def process_y(self) -> Tensor:
        y = list(range(0,30_000))
        random.shuffle(y)
        
        y_train = torch.tensor(y[:20_000])
        y_test = torch.tensor(y[20_000:])
        
        y_train = torch.tensor(list(range(0,12)))
        y_test = torch.tensor(list(range(0,12)))
        
        y_train = torch.stack([y_train, y_train], dim=0)
        y_test = torch.stack([y_test, y_test], dim=0)
        return y_train, y_test
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.edge_count_1:,}/{self.edge_count_2:,} edges)"
    
class SumEmbedding(object):
    def __call__(self, data):
        data.x1, data.x2 = data.x1.sum(dim=1), data.x2.sum(dim=1)
        return data

In [10]:
#sizmek_small = sizmek_df.sample(n=20_000, random_state=0)
#zync_small = zync_df.iloc[sizmek_small.index,:]

sizmek_small = sizmek_df.copy()
zync_small = zync_df.copy()

In [11]:
df.sample()

,account_id,city_code,state_code,dma_code,country_code,sizmek_ip,session_id,client,user_agent_platform,user_agent_language,user_agent_browser,zync_ip,zync_country,zync_state,zync_city,zync_lat,zync_long,url,referrer
54,19967,5072006,NE,722,US,69.30.184.6,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,sizmek,windows,missing,chrome,69.30.184.6,US,NE,Lincoln,40.8339,-96.5444,[https://www.windstream.net/?inc=513],[https://www.windstream.net/?inc=1039]


# Create Dataset

In [12]:
feature_cols = {
    "sizmek": {
        "categorical": [
            "account_id", "city_code", "state_code",
            "dma_code", "country_code", "sizmek_ip"
        ],
        "numeric": []
    },
    "zync": {
        "categorical": [
            "client", "user_agent_platform", "user_agent_language", 
            "user_agent_browser", "zync_ip", "zync_country",
            "zync_state", "zync_city"
        ],
        "numeric": ["zync_lat", "zync_long"]
    }
}

zeta_data = ZetaDataset(
    root="./data/",
    sizmek_df = sizmek_small,
    zync_df = zync_small,
    column=["url", "referrer"],
    label=["zeta_user_id", "client_id"],
    feature_cols=feature_cols,
    parse_url=False,
    transform=SumEmbedding()
)
zeta_data

Creating Sizmek graph. . .
Encoding features
Expanding X to 256
Creating edge list on url
Unique URLs: 26
Common URLs blocked: 0
Creating Zync graph. . .
Encoding features
Expanding X to 256
Creating edge list on referrer
Unique URLs: 23
Common URLs blocked: 0
Removing processed file. . .


Processing...
/tmp/ipykernel_31441/3789954831.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0
/tmp/ipykernel_31441/3789954831.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0
Done!


ZetaDataset(26/32 edges)

In [13]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--dim', type=int, default=256)
parser.add_argument('--rnd_dim', type=int, default=32)
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--num_steps', type=int, default=10)
parser.add_argument('--k', type=int, default=10)
args = parser.parse_args("")


psi_1 = dgmc.models.RelCNN(zeta_data.data.x1.size(-1), args.dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.0)
psi_2 = dgmc.models.RelCNN(args.rnd_dim, args.rnd_dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.0)

print(psi_1)
print(psi_2)
print(zeta_data.data.x1.size())
print(zeta_data.data.x2.size())

RelCNN(256, 256, num_layers=3, batch_norm=False, cat=True, lin=True, dropout=0.0)
RelCNN(32, 32, num_layers=3, batch_norm=False, cat=True, lin=True, dropout=0.0)
torch.Size([14, 256])
torch.Size([14, 256])


In [16]:
model = dgmc.models.DGMC(psi_1, psi_2, num_steps=None, k=args.k).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
data = zeta_data.data

def train():
    model.train()
    optimizer.zero_grad()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None, data.train_y)

    loss = model.loss(S_L, data.train_y)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test():
    model.eval()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None)

    hits1 = model.acc(S_L, data.test_y)
    hits10 = model.hits_at_k(10, S_L, data.test_y)

    return hits1, hits10

In [18]:
#torch.cuda.empty_cache()
#model.to(device)
#zeta_data.data.to(device)

model.num_steps = 0
with torch.profiler.profile(
    schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=6,
        repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler("./logs/gnn_v5"),
    with_stack=True
) as profiler:
    for epoch in range(1, 400):
        #print("Epoch: {}".format(epoch))
        if epoch == 100:
            print('Refine correspondence matrix...')
            model.num_steps = args.num_steps
            model.detach = True

        loss = train()
        profiler.step()
        
        if epoch % 5 == 0:
            hits1, hits10 = test()
            print((f"{epoch:03d}: Loss: {loss:.4f}, Hits@1: {hits1:.4f}, "
               f"Hits@10: {hits10:.4f}"))

005: Loss: 11.9517, Hits@1: 0.0833, Hits@10: 0.6667
010: Loss: 12.8142, Hits@1: 0.0833, Hits@10: 0.6667
015: Loss: 10.6789, Hits@1: 0.0833, Hits@10: 0.6667
020: Loss: 12.8321, Hits@1: 0.0833, Hits@10: 0.6667
025: Loss: 11.2920, Hits@1: 0.0000, Hits@10: 0.6667
030: Loss: 13.6256, Hits@1: 0.0833, Hits@10: 0.6667
035: Loss: 12.1114, Hits@1: 0.0833, Hits@10: 0.6667
040: Loss: 11.6813, Hits@1: 0.0000, Hits@10: 0.6667
045: Loss: 10.6119, Hits@1: 0.0833, Hits@10: 0.6667
050: Loss: 13.2480, Hits@1: 0.0833, Hits@10: 0.6667
055: Loss: 12.8488, Hits@1: 0.0000, Hits@10: 0.6667
060: Loss: 13.2567, Hits@1: 0.0833, Hits@10: 0.6667
065: Loss: 13.2137, Hits@1: 0.0833, Hits@10: 0.6667
070: Loss: 11.7832, Hits@1: 0.0833, Hits@10: 0.6667
075: Loss: 12.1165, Hits@1: 0.0833, Hits@10: 0.6667
080: Loss: 11.9517, Hits@1: 0.0833, Hits@10: 0.6667
085: Loss: 11.5091, Hits@1: 0.0833, Hits@10: 0.6667
090: Loss: 12.4108, Hits@1: 0.0000, Hits@10: 0.6667
095: Loss: 13.2449, Hits@1: 0.1667, Hits@10: 0.6667
Refine corre

In [20]:
model.eval()
_, S_L = model(
    data.x1, data.edge_index1, None, None, data.x2,
    data.edge_index2, None, None
)

model.acc(S_L, data.test_y)

0.0

In [26]:
S = S_L
y = data.test_y
pred = S.__idx__[y[0], S.__val__[y[0]].argmax(dim=-1)]
pred

tensor([ 8, 10, 10, 10,  8,  8,  8,  8,  6, 10,  8,  6])

In [23]:
data.test_y

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])

In [27]:
(pred == y[1]).sum().item()


0